## Keras模型
Keras的核心数据结构是**模型**，模型是一种组织网络层的方式。
Keras中主要的模型是**Sequential**模型，它是一系列网络层按顺序构成的栈。

In [2]:
from keras.models import Sequential
model = Sequential()

将一些网络层通过`.add()`堆叠起来，就构成了一个模型。

In [3]:
from keras.layers import Dense, Activation

model.add(Dense(units=64, input_dim=100))
model.add(Activation("relu"))
model.add(Dense(units=10))
model.add(Activation("softmax"))

完成模型的搭建后，使用`.compile()`方法编译模型。

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

编译模型时必须指明**损失函数**和**优化器**，也可以自己定制损失函数。例如下面的SGD函数中，lr是**学习速率**，momentum是**动量**，nesterov是一个boolean，为True时**确定使用nesterov动量**。另外还有decay参数，控制每次更新后的**学习速率衰减值**。

In [9]:
from keras.optimizers import SGD
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))

完成模型编译后，我们在训练数据上按batch进行一定次数的迭代来训练网络。

In [11]:
# model.fit(x_train, y_train, epochs=5, batch_size=32)

也可以手动将一个个batch送入网络中训练。

In [13]:
# model.train_on_batch(x_batch, y_batch)

随后可以使用一行代码对模型进行评估。

In [15]:
# loss_and_matrics = model.evaluate(x_test, y_test, batch_size=128)

使用搭建好的模型对新的数据进行预测。

In [17]:
# class = model.predict(x_test, batch_size=128)

## Keras中的一些基本概念
### 张量
张量可以看作是向量、矩阵的自然推广，我们用张量来表示广泛的数据类型。

In [19]:
import numpy as np
a = np.array([[1, 2], [3, 4]])
sum0 = np.sum(a, axis=0)
sum1 = np.sum(a, axis=1)

print(sum0)
print(sum1)

[4 6]
[3 7]


### data_format
在如何表示一组彩色图片上，Theano和Tensorflow发生了分歧。

+ Theano或Caffe模式：(100, 3, 32, 32)。第0个维度是样本维，代表样本的数目，第1个维度是通道维，代表颜色通道数。后面两个是高和宽，这种数据组织方法称为`channels_first`。

+ Tensorflow模式：(100, 32, 32, 3)。通道维被放在最后，称为`channels_last`。

## Keras常见问题

### 如何用Keras调用多个GPU？
有多个GPU时，推荐使用TensorFlow作为后端。有两种方法可以在多张GPU上运行一个模型：**数据并行**和**设备并行**。大多数情况下使用数据并行。

#### 数据并行
将目标模型在多个设备上各复制一份，并使用每个设备上的复制品处理整个数据集的不同部分数据。可以使用`keras.utils.multi_gpu_model`进行。最高支持在8片GPU上并行。

In [22]:
from keras.utils import multi_gpu_model

parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.complie(loss='categorical_crossentropy', optimizer='rmsprop')

# batch_size设置为256，每个GPU上将会处理32个samples
parallel_model.fit(x, y, epochs=20, batch_size=256)

AttributeError: 'Session' object has no attribute 'list_devices'

#### 设备并行
设备并行是在不同设备上运行同一个模型的不同部分，当模型含有多个并行结构，例如含有两个分支时，这种方法很适合。

这种并行方法可以通过`tf.device`实现。

In [28]:
import tensorflow as tf
import keras

input_a = keras.Input(shape=(140, 256))
input_b = keras.Input(shape=(140, 256))

shared_lstm = keras.layers.LSTM(64)

# Process the first sequence on one GPU
with tf.device('/gpu:0'):
    encoded_a = shared_lstm(tweet_a)
    
# Process the next sequence on another GPU
with tf.device('/gpu:1'):
    encoded_b = shared_lstm(tweet_b)

# Concatenate results on CPU
with tf.device('/cpu:0'):
    merged_vector = keras.layers.concatenate([encoded_a, encoded_b],
                                             axis=-1)

NameError: name 'tweet_a' is not defined

### 如何保存Keras模型
不推荐使用pickle或者cPickle保存Keras模型

可以使用`model.save(filepath)`将Keras模型和权重保存在一个HDF5文件中，该文件包含：

    + 模型的结构
    + 模型的权重
    + 训练配置（损失函数，优化器等）
    + 优化器的状态，以便于从上次训练终端的地方开始
    
使用`keras.models.load_model(filepath)`重新实例化模型，如果文件中存储了训练配置，该函数还会同时完成模型的编译。

In [29]:
from keras.models import load_model

model.save('my_model.h5')
del model

model = load_model('my_model.md5')

OSError: Unable to open file (Unable to open file: name = 'my_model.md5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

如果只是希望保存模型的结构，而不包含其权重或配置信息，可以使用JSON或YAML保存。

In [31]:
json_string = model.to_json()

yaml_string = model.to_taml()

from keras.models import model_from_json
model = model_from_json(json_string)

from keras.models import model_from_yaml
model = model_from_yaml(yaml_string)

NameError: name 'model' is not defined

如果需要保存模型的权重，可通过下面的代码利用HDF5进行保存。注意，在使用前需要确保你已安装了HDF5和其Python库h5py。

In [32]:
model.save_weights('my_model_weights.h5')

# load
model.load_weights('my_model_weights.h5')

NameError: name 'model' is not defined

如果你需要加载权重到不同的网络结构（有些层一样）中，例如fine-tune或transfer-learning，你可以通过层名字来加载模型。例如：

In [34]:
"""
假如原模型为：
    model = Sequential()
    model.add(Dense(2, input_dim=3, name="dense_1"))
    model.add(Dense(3, name="dense_2"))
    ...
    model.save_weights(fname)
"""
# new model
model = Sequential()
model.add(Dense(2, input_dim=3, name="dense_1"))  # will be loaded
model.add(Dense(10, name="new_dense"))  # will not be loaded

# load weights from first model; will only affect the first layer, dense_1.
model.load_weights('my_model_weights.h5', by_name=True)

OSError: Unable to open file (Unable to open file: name = 'my_model_weights.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

### 如何获取中间层输出
#### 创建一个新的Model
一种简单的方法是创建一个新的Model，使它的输出是你想要的那个输出

In [36]:
from keras.models import Model

model = ...

layer_name = 'my_layer'
intermediate_layer_model = Model(input=model.input, output=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(data)

AttributeError: 'ellipsis' object has no attribute 'input'

#### 建立一个Keras函数

In [38]:
from keras import backend as K

get_3rd_layer_output = K.function([model.layers[0].input],
                                 [model.layers[3].output])
layer_outupt = get_3rd_layer_output([X])[0]

AttributeError: 'ellipsis' object has no attribute 'layers'

如果你的模型在训练和测试两种模式下不完全一致，例如你的模型中含有Dropout层，批规范化（BatchNormalization）层等组件，你需要在函数中传递一个learning_phase的标记，像这样：

In [39]:
get_3rd_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[3].output])

# output in test mode = 0
layer_output = get_3rd_layer_output([X, 0])[0]

# output in train mode = 1
layer_output = get_3rd_layer_output([X, 1])[0]

AttributeError: 'ellipsis' object has no attribute 'layers'

### 当验证集的loss不再下降时，如何中断训练？
可以定义`EarlyStopping`来提前终止训练

In [40]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(X, y, validation_split=0.2, callbacks=[early_stopping])

AttributeError: 'ellipsis' object has no attribute 'fit'

### 如何在每个epoch后记录训练/测试的loss和正确率？
model.fit在运行结束后返回一个History对象，其中含有的history属性包含了训练过程中损失函数的值以及其他度量指标。

In [41]:
hist = model.fit(X, y, validation_split=0.2)
print(hist.history)

AttributeError: 'ellipsis' object has no attribute 'fit'